![LogoUC3M](https://upload.wikimedia.org/wikipedia/commons/thumb/a/a6/Acr%C3%B3nimo_y_nombre_de_la_UC3M.svg/320px-Acr%C3%B3nimo_y_nombre_de_la_UC3M.svg.png)

*Alonso Rios Guerra - 100495821 | Guillermo Sancho González - 100495991*


# *__Aprendizaje automático P1: Predicción del abandono de empleados__*

## *__1. Introducción__*

En esta práctica tenemos como objetivo desarrollar diferentes métodos de aprendizaje automático para predecir el abandono de los trabajadores de una empresa.

Primero de todo empezaremos leyendo los datos que nos proporciona la empresa. En nuestro caso, usaremos el dataset Nº10.

In [52]:
import numpy as np
import pandas as pd

data_path = 'attrition_availabledata_10.csv.gz'

data = pd.read_csv(data_path, compression='gzip', sep = ',')

## *__2. EDA Simplificado__*



Un EDA es una análisis exploratorio de datos, para organizar los datos, entender su contenido, entender cual son las variables más relevantes y cómo se relacionan unas con otras, determinar qué hacer con los datos faltantes y con los datos atípicos, y finalmente extraer conclusiones acerca de todo este análisis.

Para hacer un eda debemos responder a distintas preguntas:

-   ¿Cuántas instancias y atributos hay?

-   ¿Qué tipo de atributos hay (numéricos o categóricos)? Esto se hace para verificar si hay características categóricas que deben ser codificadas (como variables dummy o one-hot encoding). Comprobar si hay variables categóricas con alta cardinalidad.

-   ¿Qué atributos tienen valores faltantes y cuántos?

-   ¿Existen columnas constantes o ID?

-   ¿Es un problema de clasificación o regresión (variable de respuesta) y? En caso de clasificación, ¿las clases están desbalanceadas?

A continuación le damos respuesta:

-   ¿Cuántas instancias y atributos hay?

In [53]:
print('La forma de la tabla es:', data.shape)

La forma de la tabla es: (2940, 31)


El dataset contiene 2940 instancias, 30 atributos y 1 etiqueta (Attrition).

-   ¿Qué tipo de atributos hay (numéricos o categóricos)? Esto se hace para verificar si hay características categóricas que deben ser codificadas (como variables dummy o one-hot encoding). Comprobar si hay variables categóricas con alta cardinalidad.

In [54]:
print('Los tipos de atributos son:')
print('==================================')
print(data.dtypes)

Los tipos de atributos son:
hrs                        float64
absences                     int64
JobInvolvement               int64
PerformanceRating            int64
EnvironmentSatisfaction    float64
JobSatisfaction            float64
WorkLifeBalance            float64
Age                          int64
BusinessTravel              object
Department                  object
DistanceFromHome             int64
Education                    int64
EducationField              object
EmployeeCount                int64
EmployeeID                   int64
Gender                      object
JobLevel                     int64
JobRole                     object
MaritalStatus               object
MonthlyIncome                int64
NumCompaniesWorked         float64
Over18                      object
PercentSalaryHike            int64
StandardHours                int64
StockOptionLevel             int64
TotalWorkingYears          float64
TrainingTimesLastYear        int64
YearsAtCompany             

Existen dos tipos de atributos en nuestro dataset: numéricos y categóricos. Dentro de los numéricos encontramos de tipo entero (absences, Age, JobLevel, ...) y de tipo float (hrs, TotalWorkingYears, JobSatisfaction). En cuanto a lo atributos categóricos encontramos algunos como Department, JobRole, MaritalStatus, ... Para entrenar a nuestro modelo nos interesa codificar las variables categóricas y por ello es importante ver como de viable es según su cardinalidad.

In [55]:
columnas_cat = data.select_dtypes(include=['object']).columns # Selecciona las columnas categóricas

print('Cardinalidad de los atributos categóricos:')
print('================================')
for col in columnas_cat: # Imprime la cardinalidad de cada atributo categórico
    print(f"{col}: {data[col].nunique()} categorías únicas")

Cardinalidad de los atributos categóricos:
BusinessTravel: 3 categorías únicas
Department: 3 categorías únicas
EducationField: 6 categorías únicas
Gender: 2 categorías únicas
JobRole: 9 categorías únicas
MaritalStatus: 3 categorías únicas
Over18: 1 categorías únicas
Attrition: 2 categorías únicas


Al observar la ejecución del código anterior vemos que la cardinalidad de nuestros atributos categóricos es baja, en un rango de [2-9], y por ello no implicará ningún problema a la hora realizar una codificación dummy o One-Hot Encoding.

-   ¿Qué atributos tienen valores faltantes y cuántos?

In [56]:
print('Cuántos valores faltan por atributo:')
print('====================================')
sin_valor = data.isnull().sum()  # Cuenta valores nulos por columna
sin_valor = sin_valor[sin_valor > 0]  # Filtra solo los que tienen valores nulos

print(sin_valor)

Cuántos valores faltan por atributo:
EnvironmentSatisfaction    15
JobSatisfaction            12
WorkLifeBalance            29
NumCompaniesWorked         17
TotalWorkingYears           5
dtype: int64


Tras ejecutar el código anterior, obtenemos que existen 5 atributos con valores faltantes. Estos atributos son:
EnvironmentSatisfaction con 15 faltantes,
JobSatisfaction con 12 faltantes,
WorkLifeBalance con 29 faltantes,
NumCompaniesWorked con 17 faltantes y
TotalWorkingYears con 5 faltantes.

- ¿Existen columnas constantes o ID?

In [57]:
# 1. Comprobar columnas constantes
constantes = [col for col in data.columns if data[col].nunique() == 1]
print("Columnas constantes:", constantes)

# 2. Comprobar columnas ID
columnas_id = [col for col in data.columns if data[col].nunique() == len(data)]
print("Columnas ID:", columnas_id)


Columnas constantes: ['EmployeeCount', 'Over18', 'StandardHours']
Columnas ID: ['EmployeeID']


Observamos que existen 3 columnas constantes (EmployeeCount, Over18, StandardHours) y una columna ID (EmployeeID)

- ¿Es un problema de clasificación o regresión (variable de respuesta) y? En caso de clasificación, ¿las clases están desbalanceadas?


En este caso es fácil ver que es un problema de __clasificación__ porque la etiqueta (Attrition) en los datos train solo pueden tener valores 'Yes' o 'No', por lo que es una clase binaria.

In [58]:
print('Comprobar si la clase está desbalanceada:')
print('======================================')
print(data['Attrition'].value_counts())
print()
print(data['Attrition'].value_counts() / data['Attrition'].count())

Comprobar si la clase está desbalanceada:
Attrition
No     2466
Yes     474
Name: count, dtype: int64

Attrition
No     0.838776
Yes    0.161224
Name: count, dtype: float64


Se puede ver que la clase esta bastante desbalanceada: 83.88% No, 16.12% Yes

## *__3. ¿Cómo se va a realizar la evaluación?__*

Para realizar la evaluación de nuestro modelo vamos a seguir una serie de pasos. La evaluación estará divida en inner, donde se elegirá el mejor classifier con el mejor scaler, imputer y ajuste de hiperparámetros, y outer, donde se estimará el rendimiento a futuro del modelo. Es importante dividir nuestro dataset en datos de entrenamiento (usados en inner) y de validación (usados en outer). En nuestro caso seguiremos el Holdout ((2/3) Train y (1/3) Test).

- Inner Evaluation

    - Primero, determinamos el mejor scaler e imputer. Para ello compararemos el score de hacer la cross-validation con 3 folds para KNN con los hiperparámetros por defecto variando el scaler (Standard, MinMax y Robust) y el imputer (Mean y Median).

    - Una vez obtenido el mejor scaler e imputer, procederemos a buscar el mejor modelo. Para ello compararemos la precision de distintos modelos como KNN, Trees y Linear y SVM. Cada uno se comprobara con los hiper parámetros default y con los hiper parámetros optimizados.

    - Con el mejor modelo elegido, pasaremos a la Outer Evaluation.

- Outer Evaluation

    - Con el mejor modelo obtenido en la fase de evaluación interna, validaremos su entrenamiento haciendo uso de la partición de datos TEST (1/3). Esto nos permitirá estimar el rendimiento del modelo elegido con vista a futuro.

Además, una vez realizadas ambas evaluaciones, someteremos a nuestro modelo a unos datos de competición que nos devolverán ciertas predicciones.

## *__4. Metodos básicos: KNN y Tree__*

Inicialmente vamos a eliminar las columnas constantes o ids que consideramos que no aportan información útil al modelo.

In [59]:
data = data.drop(columns=constantes + columnas_id)

Se dividen los datos en x e y, donde x son los inputs, e y es la etiqueta

Despues se vuelven a dividir en train y en test. Train contiene el 66% de los datos y se usará para la evaluación interna, y test contiene el 33% de los datos y se usará para la evaluación final. Los valores estarán estratificados porque las clases están muy desbalanceadas, es decir, la proporción de positivos y negativos en train y test será igual que la del conjunto de datos original.


In [60]:
from sklearn.model_selection import train_test_split


X = data.drop(columns=['Attrition'])
y = data['Attrition']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=1/3, random_state=42, stratify= y)

Se itera creando pipelines con los distintos imputers y scalers para observar cual es el que tiene más precisión en el modelo.

Se busca que haya una accuracy mínima de 0,8388 porque al estar las clases tan desbalanceadas esa sería la tasa de aciertos de un clasificador "dummy".
En el caso de la balanced accuracy, se busca también que sea mayor a 0,5 que sería el valor del dummy classifier

In [61]:
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import OneHotEncoder, StandardScaler, MinMaxScaler, RobustScaler
from sklearn.impute import SimpleImputer
from sklearn.neighbors import KNeighborsClassifier
from sklearn.compose import ColumnTransformer
from sklearn.model_selection import StratifiedKFold, cross_val_score, cross_val_predict
from sklearn.metrics import confusion_matrix

#Separar variables categoricas y numéricas
columnas_num = X.select_dtypes(include=['float64', 'int64']).columns.tolist()
columnas_cat = X.select_dtypes(include=['object']).columns.tolist()


#Distintos métodos de escalado e imputación
scalers = [StandardScaler(), MinMaxScaler(), RobustScaler()]
imputers = ['mean', 'median']

#Se realiza una crossvalidation estratificada con 3 folds
cv = StratifiedKFold(n_splits=3, shuffle=True, random_state=42)

accuracy = -np.inf
best_scaler, best_imputer = None, None

# Definir los pasos en la Pipeline
knn = KNeighborsClassifier()
encoder = OneHotEncoder()

# Pipeline para valores categóricos
classif_categoricos = Pipeline([
            ("encoder", encoder),
            ("imputation", SimpleImputer(strategy='most_frequent'))
        ])

for i in range(len(imputers)):
    for j in range(len(scalers)):
        scaler = scalers[j]
        imputer = SimpleImputer(strategy=imputers[i])

        classif_numericos = Pipeline([
            ("imputation", imputer),
            ("standardization", scaler)
        ])

        preprocessor = ColumnTransformer(
            transformers=[
                ("num", classif_numericos, columnas_num),
                ("cat", classif_categoricos, columnas_cat)
            ]
        )

        clf = Pipeline([("preprocessor", preprocessor), ("classifier", knn)])

        clf.fit(X_train, y_train)

        scr = cross_val_score(clf, X_train, y_train, scoring='balanced_accuracy', cv=cv)

        print(f"Resultado de cross_val_score = {scr.mean()}")

        if scr.mean() > accuracy:
            accuracy = scr.mean()
            best_scaler, best_imputer = scalers[j], imputers[i]

        """
        # Calcular matriz de confusión para ver el TPR y TNR
        y_pred = cross_val_predict(clf, X_train, y_train, cv=cv)

        tn, fp, fn, tp = confusion_matrix(y_train, y_pred).ravel()

        # Calculamos TPR, TNR, Accuracy y Balanced Accuracy 
        tpr = tp / (tp + fn)
        tnr = tn / (tn + fp)

        acc = (tp + tn) / (tp + fp + tn + fn)
        bal_acc = (tpr + tnr) / 2

        print(f"Matriz de confusión: tpr: {tpr}, tnr: {tnr}")
        print(f"Balanced accuracy: {bal_acc:.4f}")
        print(f"Accuracy: {acc:.4f}\n\n")
        """

print(f"Best balanced accuracy: {accuracy:.4f}. With scaler: {best_scaler}, imputer: {best_imputer}")

Resultado de cross_val_score = 0.5903895676619819
Resultado de cross_val_score = 0.5947864542686192
Resultado de cross_val_score = 0.5954556086769281
Resultado de cross_val_score = 0.5903895676619819
Resultado de cross_val_score = 0.5947864542686192
Resultado de cross_val_score = 0.5948473361708454
Best balanced accuracy: 0.5955. With scaler: RobustScaler(), imputer: mean


El mejor scaler e imputer son ...

Una vez sabemos cual es el mejor scaler y el mejor imputer, hacemos el mismo proceso hecho antes, pero para elegir los mejores hiperparámetros del KNN.

In [ ]:
from sklearn.model_selection import GridSearchCV

classif_numericos = Pipeline([
    ("imputation", SimpleImputer(strategy=best_imputer)),
    ("standardization", best_scaler)
])

preprocessor = ColumnTransformer(
    transformers=[
        ("num", classif_numericos, columnas_num),
        ("cat", classif_categoricos, columnas_cat)
    ]
)

clf = Pipeline([("preprocessor", preprocessor), ("classifier", knn)])

hpo_grid = {"classifier__n_neighbors": [1, 2, 3, 4, 8, 16, 32], "classifier__metric": ["euclidean", "manhattan"]}

grid = GridSearchCV(clf, hpo_grid, cv=cv, scoring='balanced_accuracy')
grid.fit(X_train, y_train)
knn_hpo = grid.best_score_
print(f"knn con HPO = {knn_hpo}")
print("Mejores hiperparámetros:", grid.best_params_)




"""
# Calcularlo manualmente

accuracy = -np.inf
best_n, best_p = 0, 0

for n in [1, 2, 3, 4, 8, 16, 32]: # Elegir el numero de neighbors
    for p in [1, 2]: # Usar la distancia de Manhattan (1) o Euclídea (2)

        clf = Pipeline([("preprocessor", preprocessor), ("classifier", KNeighborsClassifier(p=p, n_neighbors=n))])

        y_pred = cross_val_predict(clf, X_train, y_train, cv=cv)

        tn, fp, fn, tp = confusion_matrix(y_train, y_pred).ravel()

        # Calculamos TPR, TNR, Accuracy y Balanced Accuracy 
        tpr = tp / (tp + fn)
        tnr = tn / (tn + fp)

        bal_acc = (tpr + tnr) / 2
        acc = (tp + tn) / (tp + fp + tn + fn)
        
        if bal_acc > accuracy:
            accuracy = bal_acc
            best_n, best_p = n, p
        
        print(f"Matriz de confusión: tpr: {tpr}, tnr: {tnr}")
        print(f"Balanced accuracy: {bal_acc:.4f}")
        print(f"Accuracy: {acc:.4f}")

print(f"Best balanced accuracy: {accuracy:.4f}. With n: {best_n}, p: {best_p}")

"""

knn con HPO = 0.7874298546485573


'\n# Calcularlo manualmente\n\naccuracy = -np.inf\nbest_n, best_p = 0, 0\n\nfor n in [1, 2, 3, 4, 8, 16, 32]: # Elegir el numero de neighbors\n    for p in [1, 2]: # Usar la distancia de Manhattan (1) o Euclídea (2)\n\n        clf = Pipeline([("preprocessor", preprocessor), ("classifier", KNeighborsClassifier(p=p, n_neighbors=n))])\n\n        y_pred = cross_val_predict(clf, X_train, y_train, cv=cv)\n\n        tn, fp, fn, tp = confusion_matrix(y_train, y_pred).ravel()\n\n        # Calculamos TPR, TNR, Accuracy y Balanced Accuracy \n        tpr = tp / (tp + fn)\n        tnr = tn / (tn + fp)\n\n        bal_acc = (tpr + tnr) / 2\n        acc = (tp + tn) / (tp + fp + tn + fn)\n        \n        if bal_acc > accuracy:\n            accuracy = bal_acc\n            best_n, best_p = n, p\n        \n        print(f"Matriz de confusión: tpr: {tpr}, tnr: {tnr}")\n        print(f"Balanced accuracy: {bal_acc:.4f}")\n        print(f"Accuracy: {acc:.4f}")\n\nprint(f"Best balanced accuracy: {accuracy:.4

Los hiperparámetros optimizados obtenidos son que el valor de K (n-neighbors) sea 1 y se utilize la distancia de Manhattan (p=1).

Se observa que cuantos menos neighbors haya, mayor sera el TPR y menor será el TNR, pero cuantos más neighbors haya, será al revés.
Como la clase minoritaria es la positiva, estamos más intereresados en los que acierten más ésta, es decir, mayor TPR.
El problema es que si se selecciona un valor de k muy bajo, el modelo se ha sobreadaptado a los datos train, y habrá overfitting.

Decision Tree con los hiperparámetros por defecto

In [63]:
from sklearn import tree

clf = Pipeline([("preprocessor", preprocessor), ("classifier", tree.DecisionTreeClassifier())])

clf.fit(X_train, y_train)

#Para ver el arbol 
"""
import matplotlib.pyplot as plt
fig = plt.figure(figsize=(200,200))
_ = tree.plot_tree(clf.named_steps["classifier"],
                feature_names = clf.named_steps["preprocessor"].get_feature_names_out(),
                class_names= ['No', 'Yes'],
                filled=True)
"""

scr = cross_val_score(clf, X_train, y_train, scoring='balanced_accuracy', cv=cv)
print(f"Resultado de cross_val_score = {scr.mean()}")

"""
# Calcular matriz de confusión

y_pred = cross_val_predict(clf, X_train, y_train, cv=cv)
tn, fp, fn, tp = confusion_matrix(y_train, y_pred).ravel()

# Calculamos TPR, TNR, Accuracy y Balanced Accuracy 
tpr = tp / (tp + fn)
tnr = tn / (tn + fp)

bal_acc = (tpr + tnr) / 2
acc = (tp + tn) / (tp + fp + tn + fn)

print(f"Matriz de confusión: tpr: {tpr}, tnr: {tnr}")
print(f"Balanced accuracy: {bal_acc:.4f}")
print(f"Accuracy: {acc:.4f}")
"""

Resultado de cross_val_score = 0.7712584409423687


'\n# Calcular matriz de confusión\n\ny_pred = cross_val_predict(clf, X_train, y_train, cv=cv)\ntn, fp, fn, tp = confusion_matrix(y_train, y_pred).ravel()\n\n# Calculamos TPR, TNR, Accuracy y Balanced Accuracy \ntpr = tp / (tp + fn)\ntnr = tn / (tn + fp)\n\nbal_acc = (tpr + tnr) / 2\nacc = (tp + tn) / (tp + fp + tn + fn)\n\nprint(f"Matriz de confusión: tpr: {tpr}, tnr: {tnr}")\nprint(f"Balanced accuracy: {bal_acc:.4f}")\nprint(f"Accuracy: {acc:.4f}")\n'

Ajuste de hiperparámetros para el arbol:

Primero elegimos el criterio utilizado

In [64]:


tree_grid = {
    'classifier__criterion': ['gini', 'entropy'],  # Función de evaluación
    'classifier__max_depth': [1, 2, 4, 8, 16, None],  # Profundidad máxima del árbol
    'classifier__min_samples_split': [2, 10, 20, 30, 100],  # Mínimo de muestras para dividir un nodo
    'classifier__min_impurity_decrease': [0.0, 0.01, 0.05]  # Umbral para crear nuevas divisiones
}

grid = GridSearchCV(clf, tree_grid, cv=cv, scoring='balanced_accuracy')
grid.fit(X_train, y_train)
knn_hpo = grid.best_score_
print(f"knn con HPO = {knn_hpo}")


knn con HPO = 0.767149732316376


In [65]:

for criterion in ["gini", "entropy"]:
    clf = Pipeline([("preprocessor", preprocessor), ("classifier", tree.DecisionTreeClassifier(criterion=criterion))])

    clf.fit(X_train,y_train)
    y_pred = cross_val_predict(clf, X_train, y_train, cv=cv)

    tn, fp, fn, tp = confusion_matrix(y_train, y_pred).ravel()

    # Calculamos TPR, TNR, Accuracy y Balanced Accuracy 
    tpr = tp / (tp + fn)
    tnr = tn / (tn + fp)

    bal_acc = (tpr + tnr) / 2
    acc = (tp + tn) / (tp + fp + tn + fn)

    print(f"Matriz de confusión: tpr: {tpr}, tnr: {tnr}")
    print(f"Balanced accuracy: {bal_acc:.4f}")
    print(f"Accuracy: {acc:.4f}")

Matriz de confusión: tpr: 0.5981012658227848, tnr: 0.9087591240875912
Balanced accuracy: 0.7534
Accuracy: 0.8587
Matriz de confusión: tpr: 0.5886075949367089, tnr: 0.920316301703163
Balanced accuracy: 0.7545
Accuracy: 0.8668


El mejor criterio es gini

Comprobando la profundidad máxima:

In [66]:
for max_depth in [1, 2, 4, 8, 16, None]:
    clf = Pipeline([("preprocessor", preprocessor), ("classifier", tree.DecisionTreeClassifier(max_depth=max_depth))])

    clf.fit(X_train,y_train)
    y_pred = cross_val_predict(clf, X_train, y_train, cv=cv)

    tn, fp, fn, tp = confusion_matrix(y_train, y_pred).ravel()

    # Calculamos TPR, TNR, Accuracy y Balanced Accuracy 
    tpr = tp / (tp + fn)
    tnr = tn / (tn + fp)

    bal_acc = (tpr + tnr) / 2
    acc = (tp + tn) / (tp + fp + tn + fn)

    print(f"Matriz de confusión: tpr: {tpr}, tnr: {tnr}")
    print(f"Balanced accuracy: {bal_acc:.4f}")
    print(f"Accuracy: {acc:.4f}")

Matriz de confusión: tpr: 0.0, tnr: 1.0
Balanced accuracy: 0.5000
Accuracy: 0.8388
Matriz de confusión: tpr: 0.23734177215189872, tnr: 0.9458637469586375
Balanced accuracy: 0.5916
Accuracy: 0.8316
Matriz de confusión: tpr: 0.26582278481012656, tnr: 0.9440389294403893
Balanced accuracy: 0.6049
Accuracy: 0.8347
Matriz de confusión: tpr: 0.4588607594936709, tnr: 0.9373479318734793
Balanced accuracy: 0.6981
Accuracy: 0.8602
Matriz de confusión: tpr: 0.6139240506329114, tnr: 0.9087591240875912
Balanced accuracy: 0.7613
Accuracy: 0.8612
Matriz de confusión: tpr: 0.6392405063291139, tnr: 0.9093673965936739
Balanced accuracy: 0.7743
Accuracy: 0.8658


Aparentemente, cuanto mayor sea la profundidad máxima, mejor precisión tendrá el modelo.

Comprobamos los efectos de modificar min_samples:

In [67]:
for min_samples in [2, 10, 20, 30, 100]:
    clf = Pipeline([("preprocessor", preprocessor), ("classifier", tree.DecisionTreeClassifier(min_samples_split=min_samples))])

    clf.fit(X_train,y_train)
    y_pred = cross_val_predict(clf, X_train, y_train, cv=cv)

    tn, fp, fn, tp = confusion_matrix(y_train, y_pred).ravel()

    # Calculamos TPR, TNR, Accuracy y Balanced Accuracy 
    tpr = tp / (tp + fn)
    tnr = tn / (tn + fp)

    bal_acc = (tpr + tnr) / 2
    acc = (tp + tn) / (tp + fp + tn + fn)

    print(f"Matriz de confusión: tpr: {tpr}, tnr: {tnr}")
    print(f"Balanced accuracy: {bal_acc:.4f}")
    print(f"Accuracy: {acc:.4f}")

Matriz de confusión: tpr: 0.6139240506329114, tnr: 0.9215328467153284
Balanced accuracy: 0.7677
Accuracy: 0.8719
Matriz de confusión: tpr: 0.5189873417721519, tnr: 0.9105839416058394
Balanced accuracy: 0.7148
Accuracy: 0.8474
Matriz de confusión: tpr: 0.4620253164556962, tnr: 0.9142335766423357
Balanced accuracy: 0.6881
Accuracy: 0.8413
Matriz de confusión: tpr: 0.43670886075949367, tnr: 0.920316301703163
Balanced accuracy: 0.6785
Accuracy: 0.8423
Matriz de confusión: tpr: 0.2563291139240506, tnr: 0.9562043795620438
Balanced accuracy: 0.6063
Accuracy: 0.8434


El mejor valor para min_samples será el que está por defecto, 2.

Finalmente, revisemos otro hiperparámetro llamado min_impurity_decrease: esto significa que solo se crea un nuevo nivel del árbol si la ganancia de información (es decir, la disminución de entropía o Gini) es mayor que min_impurity_decrease. Es otra forma de controlar la profundidad del árbol.

In [68]:
from sklearn import metrics

for min_impurity_decrease in np.linspace(0,2,num=10):
    Pipeline([("preprocessor", preprocessor), ("classifier", tree.DecisionTreeClassifier(min_impurity_decrease=min_impurity_decrease))])
    clf.fit(X_train,y_train)
    y_test_pred = clf.predict(X_test)
    accuracy_tree = metrics.accuracy_score(y_test, y_test_pred)

    print(f"With min_impurity_decrease {min_impurity_decrease}: {accuracy_tree}")


With min_impurity_decrease 0.0: 0.8448979591836735
With min_impurity_decrease 0.2222222222222222: 0.8448979591836735
With min_impurity_decrease 0.4444444444444444: 0.8448979591836735
With min_impurity_decrease 0.6666666666666666: 0.8448979591836735
With min_impurity_decrease 0.8888888888888888: 0.8448979591836735
With min_impurity_decrease 1.1111111111111112: 0.8448979591836735
With min_impurity_decrease 1.3333333333333333: 0.8448979591836735
With min_impurity_decrease 1.5555555555555554: 0.8448979591836735
With min_impurity_decrease 1.7777777777777777: 0.8448979591836735
With min_impurity_decrease 2.0: 0.8448979591836735


La modificación de éste valor parece no modificar mucho el modelo.